https://docs.google.com/spreadsheets/d/1RlPPoT6HQkQbo_607QJRJPetRsBjRhnD97N01EgK9x0/edit#gid=0

In [10]:
# !pip3 install gspread
# !pip3 install --upgrade google-api-python-client oauth2client 
# !pip3 install playsound
# !pip3 install -U discord.py
# !pip3 install dotenv

In [11]:
import requests 
import json
import gspread

import random
import os
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from playsound import playsound

import time

In [12]:
def date_to_url(s):
    """ input 17/05/1996 
        return 19962F052F17 (url format)"""
    l = s.split('/')
    if len(l)==2:
        year=2022
        day,month = l
    elif len(l)==3:
        day,month,year = l

    return f'{year}%2F{month}%2F{day}'
def time_to_int(date):
    return int(date.split(':')[0])
def get_number_of_row(sh):
    return len(sh.col_values(1))

In [ ]:
while True: 
    
    gc = gspread.service_account(filename=os.path.abspath('istheretgvmax-5273da7c9b7a.json')) #Put the path of your own json google token
    sh = gc.open("is-there-tgvmax")
    worksheet_list = [sh for sh in sh.worksheets() if '@' in sh.title]

    sh=worksheet_list[0]
    row_values = range(2,get_number_of_row(sh)+1)
    for row_value in row_values:
        #TO DO : ajouter un check dans le cas ou une ligne est vide
        try:
            origine,destination,date,heure_min,heure_max,_,email_envoye,force_update=sh.row_values(row_value)
            print(origine,destination,date,heure_min,heure_max,_,email_envoye,force_update)
            request_start = 'https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card'
            full_request = f'{request_start}&refine.destination={destination}&refine.origine={origine}&refine.date={date_to_url(date)}'
            request=requests.get(full_request)
            dico = json.loads(request.text)
            voyages = dico['records']


            nb_trouve = 0
            print(full_request)
            for (i,voyage) in enumerate(voyages):
                heure_depart = time_to_int(voyage['fields']['heure_depart']) 
                if (heure_depart>=int(heure_min) and heure_depart<=int(heure_max))and voyage['fields']['od_happy_card']=='OUI' :

                    nb_trouve +=1
            print(len(voyages),nb_trouve)
            sh.update(f'F{row_value}', nb_trouve)
            
            time.sleep(random.random()*2)
            if nb_trouve>0:
                print('TGV FOUND')
               
                if email_envoye != 'TRUE':
                    playsound("/home/gauffrea/Documents/song.mp3")
                sh.update(f'G{row_value}', 'TRUE')
        except:
            pass
    time.sleep(240)

PARIS+(intramuros) AURAY 21/07/2022 0 24 0 FALSE FALSE
https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card&refine.destination=AURAY&refine.origine=PARIS+(intramuros)&refine.date=2022%2F07%2F21
0 0
PARIS+(intramuros) AURAY 22/07/2022 0 24  FALSE FALSE
https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card&refine.destination=AURAY&refine.origine=PARIS+(intramuros)&refine.date=2022%2F07%2F22
0 0
    24  FALSE FALSE
PARIS+(intramuros) BORDEAUX+ST+JEAN 29/07/2022 0 24  FALSE FALSE
https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card&refine.destination=BORDEAUX+ST+JEAN&refine.origine=PARIS+(intramuros)&refine.date=2022%2F07%2F29
0 0
PARIS+(intramuros) BIARRITZ 29/07/2022 0 24  FALSE FALSE
https://ressources.da

In [ ]:
voyages

In [ ]:
import discord
from dotenv import load_dotenv


TOKEN = os.getenv('DISCORD_TOKEN')

client = discord.Client()

@client.event
async def on_ready():
    print(f'{client.user} has connected to Discord!')

client.run(TOKEN)

In [ ]:
gc = gspread.service_account(filename=os.path.abspath('istheretgvmax-5273da7c9b7a.json')) #Put the path of your own json google token
sh = gc.open("is-there-tgvmax")
worksheet_list = [sh for sh in sh.worksheets() if '@' in sh.title]

sh=worksheet_list[0]

In [ ]:
try :
    a=cdf
except:
    b=20
b

In [ ]:
def get_non_empty_row(sh):
    None_counter = 0
    indexes = []
    i = 2
    while None_counter<5:
        print(i)
        if sh.cell(1,i).value is not None:
            indexes += [i]
        else : None_counter += 1
        i+=1 
    return indexes

In [ ]:
get_non_empty_row(sh)

In [ ]:
A

In [ ]:
len(A)

In [ ]:
A[20]